In [1]:
import sys
sys.path.append('../modules')

from datacube import read_data
from operators import *
from preprocessing import *
from eof import *
from gridding import *
from mapper import *
from scipy import interpolate

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.feature
import seaborn as sns
import pandas as pd


%load_ext autoreload
%autoreload 2

In [2]:
PATH = '/storage/data/projects/nrc/CanRCM4_large_ensemble/design_values/tas/hdd/*.nc'
#PATH = '/home/nannau/Desktop/ensembles/jan1.0p/*'
#PATH = '/storage/data/projects/nrc/CanRCM4_large_ensemble/design_values/tas/jul2.5p/*.nc'

In [3]:
ds = read_data(PATH)
dv_field = ds['dv'].values.copy()
ens = ens_flat(dv_field)
rlat, rlon = ds['rlat'].values, ds['rlon'].values

In [4]:
irlon = np.linspace(np.min(rlon), np.max(rlon), rlon.shape[0]*10)
irlat = np.linspace(np.min(rlat), np.max(rlat), rlat.shape[0]*10)

rlon_ens = np.tile(rlon, rlat.shape[0])
rlat_ens = np.repeat(rlat, rlon.shape[0])

irlon_ens = np.tile(irlon, irlat.shape[0])
irlat_ens = np.repeat(irlat, irlon.shape[0])
icoordens = np.array(list(zip(irlon_ens, irlat_ens)))

coordens = np.array(list(zip(rlon_ens, rlat_ens)))

In [24]:
mask = load_land_mask('../data/mask.nc')
mask_master = mask_land_and_nan(dv_field, mask)
idx_ = mask_land_and_nan_ens_index(mask_master)
print(mask_master.shape)
#imask = np.tile(mask, (irlon.shape[0]*irlat.shape[0]))
mask_master = np.repeat(np.repeat(mask_master, 10, axis=1), 10) # honestly don't know how this works but it does
mask_master.shape
#idx = mask_land_and_nan_ens_index(mask_master)

(1, 130, 155)


(2015000,)

In [28]:
imask = np.expand_dims(np.reshape(mask_master, (irlat.shape[0], irlon.shape[0])), axis=0)
imask.shape
idx = mask_land_and_nan_ens_index(imask)
idx.shape
idx_.shape

(9005,)

In [29]:
#fmask = interpolate.griddata(coordens, mask_master, icoordens, method = 'cubic', fill_value=False)

#imask = fmask.copy()
#imask = np.expand_dims(imask.astype(bool), axis=0)

#print(mask[0, ...].shape)

#plt.figure(figsize=(10, 10))
#plt.imshow(mask[0, ...], extent=[np.amin(rlon), np.amax(rlon), np.amin(rlat), np.amax(rlat)], cmap='cool', alpha=0.7)
#plt.imshow(imask, cmap='Greys', alpha=0.7)
#plt.gca().invert_yaxis()
#plt.savefig('/home/nannau/Desktop/maskpic.png')
#plt.show()

In [30]:
#idx = imask == True
#idx.shape
points = icoordens[idx]

In [68]:
iens = np.broadcast_to(np.reshape(imask, (irlon.shape[0]*irlat.shape[0])), (dv_field.shape[0], irlon.shape[0]*irlat.shape[0])).copy().astype(float)


In [69]:
points = icoordens[idx]

for i in range(dv_field.shape[0]):
    iens[i, idx] = interpolate.griddata(coordens[idx_], ens[i, idx_], points, method='linear')

iens.shape, ens.shape

((35, 2015000), (35, 20150))

In [74]:
nan_mask = np.apply_over_axes(
                np.logical_or.reduce,
                np.isnan(iens[:, idx]),
                (0)
)
nan_mask[0, :].shape
#iens = iens[:, ~nan_mask[0, :]]
#iens.shape

(900500,)

In [78]:
pure = iens[:, idx]
ens = pure[:, ~nan_mask[0, :]]
